In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import tokenize as tknz
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import re
import nltk

In [2]:
df = pd.read_json(path_or_buf='gazeta_train.jsonl', lines=True)

In [3]:
df.head(4)

,url,text,title,summary,date
0,https://www.gazeta.ru/financial/2011/11/30/385...,«По итогам 2011 года чистый отток может состав...,Прогноз не успевает за оттоком,"В 2011 году из России уйдет $80 млрд, считают ...",2011-11-30 18:33:39
1,https://www.gazeta.ru/business/2013/01/24/4939...,Российское подразделение интернет-корпорации G...,Google закончил поиск,"Юлия Соловьева, экс-директор холдинга «Профмед...",2013-01-24 18:20:09
2,https://www.gazeta.ru/social/2018/02/06/116393...,Басманный районный суд Москвы вечером 6 феврал...,«Фигуранты дела могут давить на свидетелей»,Суд арестовал на два месяца четверых экс-чинов...,2018-02-06 21:21:14
3,https://www.gazeta.ru/business/2013/06/21/5388...,Как повлияло вступление в ВТО на конкурентносп...,«С последних традиционно «отжимают» больше»,Мнения предпринимателей по поводу вступления в...,2013-06-21 17:43:50


In [4]:
def preprocess_txt(line):
    morpher = MorphAnalyzer()
    sw = set(get_stop_words('ru'))
    exclude = set(string.punctuation)
  
    spls = ''.join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != '']
    
    return spls

In [5]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
            
    if norm > 0:
        vector = vector / norm
        
    answers = index.get_nns_by_vector(vector, 3, )
    
    return [index_map[i] for i in answers]

In [6]:
def lower_words(text):
    lower_text = text.lower()

    return lower_text

In [7]:
sentences = [row.split() for row in df['text']]

In [10]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=1)

In [11]:
modelFT = FastText(sentences=sentences, vector_size=300, min_count=1, window=5, workers=8)

MemoryError: Unable to allocate 1.50 GiB for an array with shape (1338489, 300) and data type float32

Пока не успел доделать